In [1]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import scipy.signal
import sk_dsp_comm.pyaudio_helper as pah
import librosa
import librosa.display
from IPython.display import Audio
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pprint

In [4]:
pprint.pprint(pah.available_devices())
display(Audio('a2_mp_rr2.wav'))
display(Audio('output.wav'))
x, sr = librosa.load('a2_mp_rr2.wav')
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(10, 3))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Log-frequency power spectrogram')


{0: {'inputs': 0, 'name': 'HDA Intel HDMI: 0 (hw:0,3)', 'outputs': 8},
 1: {'inputs': 0, 'name': 'HDA Intel HDMI: 1 (hw:0,7)', 'outputs': 8},
 2: {'inputs': 0, 'name': 'HDA Intel HDMI: 2 (hw:0,8)', 'outputs': 8},
 3: {'inputs': 2,
     'name': 'HDA Intel PCH: CX20751/2 Analog (hw:1,0)',
     'outputs': 2},
 4: {'inputs': 2, 'name': 'Scarlett 2i4 USB: Audio (hw:2,0)', 'outputs': 4},
 5: {'inputs': 0, 'name': 'hdmi', 'outputs': 8},
 6: {'inputs': 32, 'name': 'pulse', 'outputs': 32},
 7: {'inputs': 32, 'name': 'default', 'outputs': 32}}


FigureCanvasNbAgg()

Text(0.5, 1.0, 'Log-frequency power spectrogram')

In [3]:
x_harm, x_perc = librosa.effects.hpss(x)
plt.figure()
librosa.display.waveplot(x_harm, sr=sr, alpha=0.25)
librosa.display.waveplot(x_perc, sr=sr, color='r', alpha=0.5)
plt.title('Harmonic + Percussive')
plt.tight_layout()

display(Audio(x_harm, rate = sr))
display(Audio(x_perc, rate = sr))


FigureCanvasNbAgg()

In [4]:
Record_gain = widgets.FloatSlider(
    description="Record Gain",
    continuous_update=True,
    value=1.0,
    min=0.0,
    max=2.0,
    step=0.01,
    orientation="horizontal",
)
Monitor_gain = widgets.FloatSlider(
    description="Monitor Gain",
    continuous_update=True,
    value=0.1,
    min=0.0,
    max=2.0,
    step=0.01,
    orientation="horizontal",
)

In [5]:
# define callback (3)
# Here we configure the callback to play back a wav file
def callback1(in_data, frame_count, time_info, status):
    global DSP_IO_1
    DSP_IO_1.DSP_callback_tic()
    # convert byte data to ndarray
    print('len(in_data): {}'.format(len(in_data)))
    in_data_nda = np.frombuffer(in_data, dtype=np.int16)
    # Ignore in_data when generating output only
    #***********************************************
    x = in_data_nda.astype(np.float32)
    y = Record_gain.value*x
    # Note wav is scaled to [-1,1] so need to rescale to int16
    #y = 32767*x.get_samples(frame_count)
    # Perform real-time DSP here if desired
    #
    #***********************************************
    # Save data for later analysis
    # accumulate a new frame of samples
    DSP_IO_1.DSP_capture_add_samples(y)
    #***********************************************
    # Monitor level control
    y *= Monitor_gain.value
    # Convert from float back to int16
    y = y.astype(np.int16)
    DSP_IO_1.DSP_callback_toc()
    return y.tobytes(), pah.pyaudio.paContinue

In [11]:
# fs, x_wav2 = ss.from_wav('Music_Test.wav')
# x_wav = (x_wav2[:,0] + x_wav2[:,1])/2
# x = pah.loop_audio(x_wav)
DSP_IO_1 = pah.DSP_io_stream(callback1,1, 0,fs=48000,Tcapture=10)
DSP_IO_1.interactive_stream(Tsec=5)
widgets.HBox([Record_gain,Monitor_gain])

interactive(children=(ToggleButtons(description=' ', index=1, options=('Start Streaming', 'Stop Streaming'), v…

In [3]:
import pyaudio
import pprint
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    pprint.pprint(p.get_device_info_by_index(i))

print()

for i in range(p.get_host_api_count()):
    pprint.pprint(p.get_host_api_info_by_index(i))


{'defaultHighInputLatency': -1.0,
 'defaultHighOutputLatency': 0.034829931972789115,
 'defaultLowInputLatency': -1.0,
 'defaultLowOutputLatency': 0.008707482993197279,
 'defaultSampleRate': 44100.0,
 'hostApi': 0,
 'index': 0,
 'maxInputChannels': 0,
 'maxOutputChannels': 8,
 'name': 'HDA Intel HDMI: 0 (hw:0,3)',
 'structVersion': 2}
{'defaultHighInputLatency': -1.0,
 'defaultHighOutputLatency': 0.034829931972789115,
 'defaultLowInputLatency': -1.0,
 'defaultLowOutputLatency': 0.008707482993197279,
 'defaultSampleRate': 44100.0,
 'hostApi': 0,
 'index': 1,
 'maxInputChannels': 0,
 'maxOutputChannels': 8,
 'name': 'HDA Intel HDMI: 1 (hw:0,7)',
 'structVersion': 2}
{'defaultHighInputLatency': -1.0,
 'defaultHighOutputLatency': 0.034829931972789115,
 'defaultLowInputLatency': -1.0,
 'defaultLowOutputLatency': 0.008707482993197279,
 'defaultSampleRate': 44100.0,
 'hostApi': 0,
 'index': 2,
 'maxInputChannels': 0,
 'maxOutputChannels': 8,
 'name': 'HDA Intel HDMI: 2 (hw:0,8)',
 'structVersi